In [62]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
from shapely.geometry import shape as s_shape

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
# path = DATA_DIR/'portland_gtfs.zip'
# feed = gt.read_gtfs(path, dist_units='ft')
feed.calendar_dates


,service_id,date,exception_type
0,9.387,20140518,1
1,C.387,20140518,1
2,I.387,20140518,1
3,U.387,20140518,1
4,k.387,20140518,1
5,2.387,20140519,1
6,7.387,20140519,1
7,A.387,20140519,1
8,H.387,20140519,1
9,W.387,20140519,1


In [50]:
path = DATA_DIR/'cairns_gtfs.zip'
feed = gt.read_gtfs(path, dist_units='km')

# Pick date
date = gt.get_first_week(feed)[0]
print('date', date)


date 20140526


In [60]:
from copy import deepcopy
g1 = feed._calendar_dates_g
g2 = deepcopy(g1)
g2


In [47]:
feed.trips['route_id'] = 'bingo'
print(feed._trips_i['route_id'].head())

feed.trips['route_id'] = 'bongo'
feed.trips = feed.trips
print(feed._trips_i['route_id'].head())


trip_id
CNS2014-CNS_MUL-Weekday-00-4165878    110-423
CNS2014-CNS_MUL-Weekday-00-4165879    110-423
CNS2014-CNS_MUL-Weekday-00-4165880    110-423
CNS2014-CNS_MUL-Weekday-00-4165881    110-423
CNS2014-CNS_MUL-Weekday-00-4165882    110-423
Name: route_id, dtype: object
trip_id
CNS2014-CNS_MUL-Weekday-00-4165878    bongo
CNS2014-CNS_MUL-Weekday-00-4165879    bongo
CNS2014-CNS_MUL-Weekday-00-4165880    bongo
CNS2014-CNS_MUL-Weekday-00-4165881    bongo
CNS2014-CNS_MUL-Weekday-00-4165882    bongo
Name: route_id, dtype: object


In [48]:
ts = gt.compute_trip_stats(feed)
ts

AttributeError: 'Feed' object has no attribute 'dist_units_out'

In [ ]:
rts = gt.compute_route_time_series(feed, ts, date)
gt.downsample(rts, freq='12H')

In [ ]:
tid = feed.trips['trip_id'].iat[0]
gt.trip_to_geojson(feed, tid, include_stops=True)

In [ ]:
route_id = feed.routes['route_id'].iat[0]
geo = gt.route_to_geojson(feed, route_id, include_stops=True)
path = DATA_DIR/'route_{!s}.geojson'.format(route_id)
with path.open('w') as tgt:
    json.dump(geo, tgt)

In [ ]:
# Add distances to feed
trips_stats = gt.compute_trip_stats(feed, compute_dist_from_shapes=True)
feed.stop_times = gt.add_dist_to_stop_times(feed, trips_stats)
feed.stop_times.T

In [ ]:
trips_stats = gt.compute_trip_stats(feed)
routes_stats = gt.compute_route_stats(feed, trips_stats, date)
routes_stats


In [ ]:
a = pd.Series([np.nan, np.nan])
a.dropna().max()
#gt.get_trips(feed, date, "07:30:00")


In [ ]:
st['departure_time'].unique

In [ ]:
tl = gt.compute_trip_locations(feed, date, ['12:00:00'])
tl

In [ ]:
freq = '12H'
f = gt.compute_route_time_series(feed, trips_stats, date, split_directions=True, freq=freq)

# cols_and_hows = [
#     ('num_trips', 'mean'),
#     ('num_trip_starts', 'sum'),
#     ('service_distance', 'sum'),
#     ('service_duration', 'sum'),
#     ]

# frames = []
# for col, how in cols_and_hows:
#     frames.append(f[col].resample(freq).agg({col: how}))    
# g = pd.concat(frames, axis=1)
# g = g.sortlevel(axis=1)

# #f = f.resample(freq).sum()

# g.columns.names = f.columns.names

# g.T
f.T

In [ ]:

# Pick screen line
path = DATA_DIR/'cairns_screen_line.geojson'
with path.open() as src:
    line = json.load(src)
    line = s_shape(line['features'][0]['geometry'])

g = gt.compute_screen_line_counts(feed, line, date)
g

In [ ]:
g.groupby(['route_id', 'orientation']).count()